In [1]:
import math
import pygame
import pymunk
import pymunk.pygame_util
from pymunk.vec2d import Vec2d
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.12.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import pymunk
import pymunk.pygame_util
from pymunk.vec2d import Vec2d
import numpy as np
from typing import Optional

def generate_color(group_id: int) -> tuple[int, int, int]:
    """
    Generate a unique color based on group_id using HSV color space
    for better visual distinction between cells.
    """
    import colorsys

    # Use golden ratio for better color distribution
    golden_ratio = 0.618033988749895
    hue = (group_id * golden_ratio) % 1.0
    saturation = 0.7 + (group_id % 3) * 0.1  # Vary saturation slightly
    value = 0.8 + (group_id % 2) * 0.2  # Vary brightness slightly

    rgb: tuple[float, float, float] = colorsys.hsv_to_rgb(hue, saturation, value)
    return int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255)


#Note that length units here are the number of spheres in the cell, TODO: implement the continuous length measurement for rendering.
class Cell:
    space: pymunk.Space
    start_pos: tuple[float, float]
    num_segments: int
    segment_radius: float
    segment_mass: float
    group_id: int
    growth_rate: float
    min_length_after_division: int
    max_length_variation: float
    base_color: Optional[tuple[int, int, int]]
    base_max_length: Optional[int]
    _from_division: Optional[bool]

    max_bend_angle: float
    base_min_length_after_division: int
    base_max_length_variation: float
    noise_strength: float
    _max_length: int


    def __init__(
            self,
            space: pymunk.Space,
            start_pos: tuple[float, float],
            num_segments: int,
            segment_radius: float,
            segment_mass: float,
            group_id: int = 0,
            growth_rate: float = 5.0,
            min_length_after_division: int = 10,
            max_length_variation: float = 0.2,
            base_color: Optional[tuple[int, int, int]] = None,
            noise_strength: float = 0.05,
            base_max_length: int = 40,
            _from_division: bool = False
    ) -> None:
        """
        Initialize a segmented (bendy) cell instance.

        Parameters
        ----------
        space : pymunk.Space
            The simulation space where the cell exists and interacts with other
            physical entities.
        start_pos : tuple[float, float]
            The starting position (x, y) for creating the first segment of the cell.
        num_segments : int
            The initial number of segments to create for this cell.
        segment_radius : float
            The radius of each segment in the cell.
        segment_mass : float
            The mass of each segment in the cell.
        group_id : int, optional
            A unique identifier for grouping or categorizing the segments in cell.
            TODO: use for lineage tracking, give daughter the mother ID
        growth_rate : float, optional
            The rate at which the cell grows over time.
        _max_length : int, optional
            The maximum allowable length of the cell in terms of the number of segments.
        min_length_after_division : int, optional
            The minimal length the cell must have after it undergoes division. Defaults to 10.
        max_length_variation : float, optional
            The percentage variation allowed in determining the maximum length of the cell.
        base_color : tuple[int, int, int], optional
            The base color of the cell when displayed in Pygame, specified as an RGB tuple. If None, a color is
            generated based on the group_id.
            TODO: switch to Pyglet
        noise_strength : float, optional
            The strength of random noisy forces added to modulate the cell's dynamics.
        base_max_length : int, optional
            An optional base value for calculating the randomised maximum length. If not
            provided, the value of max_length is used.
        _from_division : bool, optional
            Indicates whether the cell is being created as a result of division.
        """
        self.space = space
        self.start_pos = start_pos
        self.segment_radius = segment_radius
        self.segment_mass = segment_mass
        self.growth_rate = growth_rate
        self.max_bend_angle = 0.005  # 0.01 normally
        self.noise_strength = noise_strength

        self.group_id = group_id
        self.base_color = base_color if base_color else generate_color(group_id)

        # Store the original base max_length for consistent inheritance
        self.base_max_length = base_max_length

        # Always randomise from the original base, not the parent's randomised value
        variation = self.base_max_length * max_length_variation
        random_max_len = np.random.uniform(
            self.base_max_length - variation, self.base_max_length + variation
        )
        self._max_length = max(min_length_after_division * 2, int(random_max_len))

        self.min_length_after_division = min_length_after_division
        self.max_length_variation = max_length_variation

        # Rest of the existing code...
        self.bodies = []
        self.shapes = []
        self.joints = []

        self.growth_accumulator = 0.0
        self.growth_threshold = self.segment_radius / 3
        self.joint_distance = self.segment_radius / 4
        self.joint_max_force = 30000

        if not _from_division:
            for i in range(num_segments):
                self._add_initial_segment(i == 0)
            self._update_colors()

    def _add_initial_segment(self, is_first):
        """
        Adds a single segment to the cell during initialization.
        """
        moment = pymunk.moment_for_circle(
            self.segment_mass, 0, self.segment_radius
        )
        body = pymunk.Body(self.segment_mass, moment)

        if is_first:
            body.position = self.start_pos
        else:
            prev_body = self.bodies[-1]
            # Keep growth perfectly straight
            offset = Vec2d(self.joint_distance, 0).rotated(prev_body.angle)
            body.position = prev_body.position + offset

            # Add tiny random positional noise to break determinism
            noise_x = np.random.uniform(-0.1, 0.1)
            noise_y = np.random.uniform(-0.1, 0.1)
            body.position += Vec2d(noise_x, noise_y)

        shape = pymunk.Circle(body, self.segment_radius)
        shape.friction = 0.0
        shape.filter = pymunk.ShapeFilter(group=self.group_id)

        self.space.add(body, shape)
        self.bodies.append(body)
        self.shapes.append(shape)

        if not is_first:
            prev_body = self.bodies[-2]

            anchor_on_prev = (self.joint_distance / 2, 0)
            anchor_on_curr = (-self.joint_distance / 2, 0)
            pivot = pymunk.PivotJoint(
                prev_body, body, anchor_on_prev, anchor_on_curr
            )
            pivot.max_force = self.joint_max_force
            self.space.add(pivot)
            self.joints.append(pivot)

            limit = pymunk.RotaryLimitJoint(
                prev_body, body, -self.max_bend_angle, self.max_bend_angle
            )
            limit.max_force = self.joint_max_force
            self.space.add(limit)
            self.joints.append(limit)

    def apply_noise(self, dt):
        """
        NEW: Apply small random forces to all segments to simulate environmental noise
        """
        for body in self.bodies:
            # Apply tiny random forces
            force_x = np.random.uniform(-self.noise_strength, self.noise_strength)
            force_y = np.random.uniform(-self.noise_strength, self.noise_strength)
            body.force += Vec2d(force_x, force_y)

            # Also apply tiny random torques
            torque = np.random.uniform(-self.noise_strength * 0.1, self.noise_strength * 0.1)
            body.torque += torque

    def grow(self, dt):
        """
        Grows the cell by extending the last segment until a new one can be added.
        """
        if len(self.bodies) >= self._max_length or len(self.bodies) < 2:
            return

        # User change: randomized growth
        self.growth_accumulator += (
                self.growth_rate * dt * np.random.uniform(0, 4)
        )
        last_pivot_joint = self.joints[-2]
        original_anchor_x = -self.joint_distance / 2
        last_pivot_joint.anchor_b = (
            original_anchor_x - self.growth_accumulator,
            0,
        )

        if self.growth_accumulator >= self.growth_threshold:
            pre_tail_body = self.bodies[-2]
            old_tail_body = self.bodies[-1]

            last_pivot_joint.anchor_b = (original_anchor_x, 0)

            stable_offset = Vec2d(self.joint_distance, 0).rotated(
                pre_tail_body.angle
            )
            old_tail_body.position = pre_tail_body.position + stable_offset
            old_tail_body.angle = pre_tail_body.angle

            moment = pymunk.moment_for_circle(
                self.segment_mass, 0, self.segment_radius
            )
            new_tail_body = pymunk.Body(self.segment_mass, moment)

            # Keep growth direction perfectly straight
            new_tail_offset = Vec2d(self.joint_distance, 0).rotated(
                old_tail_body.angle
            )
            new_tail_body.position = old_tail_body.position + new_tail_offset

            # NEW: Add tiny random positional noise to the new segment
            noise_x = np.random.uniform(-0.1, 0.1)
            noise_y = np.random.uniform(-0.1, 0.1)
            new_tail_body.position += Vec2d(noise_x, noise_y)

            new_tail_shape = pymunk.Circle(new_tail_body, self.segment_radius)
            new_tail_shape.friction = 0.0  # User change
            new_tail_shape.filter = pymunk.ShapeFilter(group=self.group_id)

            self.space.add(new_tail_body, new_tail_shape)
            self.bodies.append(new_tail_body)
            self.shapes.append(new_tail_shape)

            anchor_on_prev = (self.joint_distance / 2, 0)
            anchor_on_curr = (-self.joint_distance / 2, 0)
            new_pivot = pymunk.PivotJoint(
                old_tail_body, new_tail_body, anchor_on_prev, anchor_on_curr
            )
            new_pivot.max_force = self.joint_max_force
            self.space.add(new_pivot)
            self.joints.append(new_pivot)

            new_limit = pymunk.RotaryLimitJoint(
                old_tail_body,
                new_tail_body,
                -self.max_bend_angle,
                self.max_bend_angle,
            )
            new_limit.max_force = self.joint_max_force
            self.space.add(new_limit)
            self.joints.append(new_limit)

            self.growth_accumulator = 0.0
            self._update_colors()

    def divide(self, next_group_id: int) -> Optional['Cell']:

        """
            If the cell is at max_length, it splits by transplanting its second
            half into a new Cell object, preserving orientation.
            """
        if len(self.bodies) < self._max_length:
            return None

        split_index = len(self.bodies) // 2
        if split_index < self.min_length_after_division or (
                len(self.bodies) - split_index
        ) < self.min_length_after_division:
            return None

        # Create daughter cell - pass the BASE max_length, not this cell's randomized one
        daughter_cell = Cell(
            space = self.space,
            start_pos = self.bodies[split_index].position,
            num_segments = 0, # Start the cell off with 0 segments and add them in manually
            segment_radius = self.segment_radius,
            segment_mass = self.segment_mass,
            group_id = next_group_id,
            growth_rate = self.growth_rate,
            base_max_length = self.base_max_length,
            min_length_after_division = self.min_length_after_division,
            max_length_variation = self.max_length_variation,
            base_color = generate_color(next_group_id),
            noise_strength=self.noise_strength,
            _from_division=True)

        # Partition the mother's parts.
        daughter_cell.bodies = self.bodies[split_index:]
        daughter_cell.shapes = self.shapes[split_index:]
        daughter_cell.joints = self.joints[split_index * 2:]

        for shape in daughter_cell.shapes:
            shape.filter = pymunk.ShapeFilter(group=next_group_id)

        connecting_joint = self.joints[(split_index - 1) * 2]
        connecting_limit = self.joints[(split_index - 1) * 2 + 1]
        self.space.remove(connecting_joint, connecting_limit)

        self.bodies = self.bodies[:split_index]
        self.shapes = self.shapes[:split_index]
        self.joints = self.joints[: (split_index - 1) * 2]

        self._update_colors()
        daughter_cell._update_colors()

        return daughter_cell

    def remove_tail_segment(self):
        """
        Safely removes the last segment of the cell.
        """
        if len(self.bodies) <= self.min_length_after_division:
            return

        tail_body = self.bodies.pop()
        tail_shape = self.shapes.pop()

        tail_joint = self.joints.pop()
        tail_limit = self.joints.pop()

        self.space.remove(tail_body, tail_shape, tail_joint, tail_limit)
        self._update_colors()

    def _update_colors(self):
        """
        Sets the head and tail colors based on the cell's base color.
        """
        if not self.shapes:
            return

        # NEW: Use the cell's base color with variations
        r, g, b = self.base_color

        # Body segments: use base color with alpha
        body_color = (r, g, b, 255)

        # Head: brighter version of base color
        head_color = (
            min(255, int(r * 1.3)),
            min(255, int(g * 1.3)),
            min(255, int(b * 1.3)),
            255
        )

        # Tail: darker version of base color
        tail_color = (
            int(r * 0.7),
            int(g * 0.7),
            int(b * 0.7),
            255
        )

        # Apply colors
        for shape in self.shapes:
            shape.color = body_color

        self.shapes[0].color = head_color  # Head
        self.shapes[-1].color = tail_color  # Tail


In [3]:
"""
Initializes Pygame and Pymunk and runs the main simulation loop.
"""
pygame.init()
screen_width, screen_height = 1200, 800
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Dividing Worm Colony Simulation")

space = pymunk.Space(threaded=True)
space.threads = 2

def setup_spatial_hash(space, colony):
    """Setup spatial hash with estimated parameters"""
    dim, count = estimate_spatial_hash_params(colony)
    space.use_spatial_hash(dim, count)
    print(f"Spatial hash enabled: dim={dim:.1f}, count={count}")
    return dim, count

def estimate_spatial_hash_params(colony):
    """Estimate good spatial hash parameters for current colony size"""
    if not colony:
        return 36.0, 1000
    
    total_segments = sum(len(cell.bodies) for cell in colony)
    segment_radius = colony[0].segment_radius if colony else 15
    
    # dim: slightly larger than segment diameter for optimal performance
    dim = segment_radius * 2 * 1.2
    
    # count: ~10x total objects, with reasonable bounds
    count = max(1000, min(100000, total_segments * 10))
    print(f"Spatial hash updated: dim={dim:.1f}, count={count}")
    return dim, count

In [4]:
# Add periodic updates in your main loop:
frame_count = 0
last_segment_count = 15  # Initial worm segments

space.iterations = 60
space.gravity = (0, 0)
space.damping = 0.5
# NEW: Camera and zoom variables
zoom_level = 1.0
camera_x, camera_y = 0, 0
min_zoom = 0.1
max_zoom = 5.0

# Create a virtual surface for drawing
virtual_surface = pygame.Surface((screen_width, screen_height))
draw_options = pymunk.pygame_util.DrawOptions(virtual_surface)
pymunk.pygame_util.positive_y_is_up = False

colony = []
next_group_id = 1

initial_worm = Cell(
    space,
    start_pos=(screen_width / 2, screen_height / 2),
    num_segments=15,
    segment_radius=15,
    segment_mass=2,
    group_id=next_group_id,
    base_max_length=60.5,  # This is now the mean length
    base_color=generate_color(next_group_id),  # NEW: Set initial color
    noise_strength=0.1,  # NEW: Small environmental noise
)
colony.append(initial_worm)
next_group_id += 1

# In your main() function, after creating initial_worm:
setup_spatial_hash(space, colony)

mouse_body = pymunk.Body(body_type=pymunk.Body.KINEMATIC)
mouse_joint = None

clock = pygame.time.Clock()
running = True

simulation_speed_multiplier = 10

Spatial hash updated: dim=36.0, count=1000
Spatial hash enabled: dim=36.0, count=1000


In [5]:
initial_worm.base_max_length

60.5

In [ ]:
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (
            event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE
        ):
            running = False
        # NEW: Handle mouse wheel for zooming
        elif event.type == pygame.MOUSEWHEEL:
            old_zoom = zoom_level
            if event.y > 0:  # Scroll up - zoom in
                zoom_level = min(zoom_level * 1.1, max_zoom)
            elif event.y < 0:  # Scroll down - zoom out
                zoom_level = max(zoom_level / 1.1, min_zoom)
            
            # Adjust camera to zoom towards mouse position
            mouse_x, mouse_y = pygame.mouse.get_pos()
            world_x = (mouse_x - screen_width/2) / old_zoom + camera_x
            world_y = (mouse_y - screen_height/2) / old_zoom + camera_y
            
            camera_x = world_x - (mouse_x - screen_width/2) / zoom_level
            camera_y = world_y - (mouse_y - screen_height/2) / zoom_level
            
        # NEW: Handle keyboard zoom controls as backup
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_EQUALS or event.key == pygame.K_PLUS:
                zoom_level = min(zoom_level * 1.2, max_zoom)
            elif event.key == pygame.K_MINUS:
                zoom_level = max(zoom_level / 1.2, min_zoom)
            elif event.key == pygame.K_r:  # Reset zoom and camera
                zoom_level = 1.0
                camera_x, camera_y = 0, 0
            elif event.key == pygame.K_j:  # NEW: Toggle joint visibility
                if hasattr(draw_options, 'show_joints'):
                    draw_options.show_joints = not draw_options.show_joints
                
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # NEW: Convert mouse position to world coordinates
            mouse_x, mouse_y = event.pos
            world_x = (mouse_x - screen_width/2) / zoom_level + camera_x
            world_y = (mouse_y - screen_height/2) / zoom_level + camera_y
            pos = Vec2d(world_x, world_y)
            
            hit = space.point_query_nearest(pos, 5/zoom_level, pymunk.ShapeFilter())
            if hit is not None and hit.shape.body.body_type == pymunk.Body.DYNAMIC:
                shape = hit.shape
                rest_point = shape.body.world_to_local(pos)
                mouse_joint = pymunk.PivotJoint(
                    mouse_body, shape.body, (0, 0), rest_point
                )
                mouse_joint.max_force = 100000
                mouse_joint.error_bias = (1 - 0.15) ** 60
                space.add(mouse_joint)
        elif event.type == pygame.MOUSEBUTTONUP:
            if mouse_joint is not None:
                space.remove(mouse_joint)
                mouse_joint = None

    # NEW: Update mouse body position in world coordinates        
    mouse_x, mouse_y = pygame.mouse.get_pos()
    world_mouse_x = (mouse_x - screen_width/2) / zoom_level + camera_x
    world_mouse_y = (mouse_y - screen_height/2) / zoom_level + camera_y
    mouse_body.position = (world_mouse_x, world_mouse_y)
    
    dt = 1.0 / 60.0

    for _ in range(simulation_speed_multiplier):
        newly_born_worms_map = {}

        for worm in colony[:]:
            worm.apply_noise(dt)  # NEW: Apply environmental noise
            worm.grow(dt)
            new_worm = worm.divide(next_group_id)
            if new_worm:
                newly_born_worms_map[new_worm] = worm
                next_group_id += 1

        if newly_born_worms_map:
            counter = 0
            for daughter, mother in newly_born_worms_map.items():
                while True:
                    overlap_found = False
                    for daughter_shape in daughter.shapes:
                        query_result = space.shape_query(daughter_shape)

                        for info in query_result:
                            if info.shape in mother.shapes:
                                mother.remove_tail_segment()
                                overlap_found = True
                                break
                        if overlap_found:
                            break

                    if not overlap_found:
                        break
                    counter += 1
                    if counter > 100:
                        break

        colony.extend(newly_born_worms_map.keys())
        space.step(dt)

    # NEW: Apply camera transform to draw options
    draw_options.transform = pymunk.Transform(
        a=zoom_level, b=0, c=0, d=zoom_level,
        tx=screen_width/2 - camera_x * zoom_level,
        ty=screen_height/2 - camera_y * zoom_level
    )

    # NEW: Hide joints when zoomed out, show when zoomed in
    if zoom_level < 0.8:  # Hide joints when zoomed out
        draw_options.flags = pymunk.pygame_util.DrawOptions.DRAW_SHAPES
    else:  # Show joints when zoomed in enough
        draw_options.flags = (
            pymunk.pygame_util.DrawOptions.DRAW_SHAPES |
            pymunk.pygame_util.DrawOptions.DRAW_CONSTRAINTS
        )

    # Clear both surfaces
    screen.fill((20, 30, 40))
    virtual_surface.fill((20, 30, 40))
    
    # Draw to virtual surface with transform
    space.debug_draw(draw_options)
    
    # Blit virtual surface to main screen
    screen.blit(virtual_surface, (0, 0))
    
    # NEW: Display zoom level and controls
    font = pygame.font.Font(None, 36)
    zoom_text = font.render(f"Zoom: {zoom_level:.2f}x", True, (255, 255, 255))
    help_text = font.render("Mouse wheel: Zoom, R: Reset, J: Toggle joints", True, (255, 255, 255))
    screen.blit(zoom_text, (10, 10))
    screen.blit(help_text, (10, 50))
    
    pygame.display.flip()
    clock.tick(60)


    frame_count += 1

    # Update spatial hash every 60 frames (1 second) if colony grew significantly
    if frame_count % 60 == 0:
        current_segment_count = sum(len(worm.bodies) for worm in colony)
        if current_segment_count > last_segment_count * 1.5:  # 50% growth
            dim, count = estimate_spatial_hash_params(colony)
            space.use_spatial_hash(dim, count)
            last_segment_count = current_segment_count

pygame.quit()

Spatial hash updated: dim=36.0, count=1000
Spatial hash updated: dim=36.0, count=1000
Spatial hash updated: dim=36.0, count=1990
Spatial hash updated: dim=36.0, count=3000
Spatial hash updated: dim=36.0, count=6910
Spatial hash updated: dim=36.0, count=14850


In [ ]:
import math
import pygame
import pymunk
import pymunk.pygame_util
from pymunk.vec2d import Vec2d
import numpy as np


def generate_color(group_id):
    """
    Generate a unique color based on group_id using HSV color space
    for better visual distinction between worms.
    """
    import colorsys
    
    # Use golden ratio for better color distribution
    golden_ratio = 0.618033988749895
    hue = (group_id * golden_ratio) % 1.0
    saturation = 0.7 + (group_id % 3) * 0.1  # Vary saturation slightly
    value = 0.8 + (group_id % 2) * 0.2       # Vary brightness slightly
    
    rgb = colorsys.hsv_to_rgb(hue, saturation, value)
    return tuple(int(c * 255) for c in rgb)


def create_open_box(space, box_x, box_y, box_width, box_height, wall_thickness=10, opening_side='right'):
    """
    Creates an open-ended box using static bodies.
    
    Args:
        space: The pymunk space
        box_x, box_y: Center position of the box
        box_width, box_height: Dimensions of the box
        wall_thickness: Thickness of the walls
        opening_side: Which side is open ('left', 'right', 'top', 'bottom')
    
    Returns:
        List of wall bodies and shapes, and box boundaries dict
    """
    walls = []
    
    # Calculate wall positions relative to box center
    half_width = box_width / 2
    half_height = box_height / 2
    half_thickness = wall_thickness / 2
    
    # Store box boundaries for cleanup detection
    box_bounds = {
        'left': box_x - half_width,
        'right': box_x + half_width,
        'top': box_y - half_height,
        'bottom': box_y + half_height,
        'opening_side': opening_side
    }
    
    # Define wall configurations: (start_pos, end_pos, description)
    wall_configs = []
    
    if opening_side != 'bottom':  # Bottom wall
        wall_configs.append((
            (box_x - half_width - half_thickness, box_y + half_height),
            (box_x + half_width + half_thickness, box_y + half_height),
            "bottom"
        ))
    
    if opening_side != 'top':  # Top wall
        wall_configs.append((
            (box_x - half_width - half_thickness, box_y - half_height),
            (box_x + half_width + half_thickness, box_y - half_height),
            "top"
        ))
    
    if opening_side != 'left':  # Left wall
        wall_configs.append((
            (box_x - half_width, box_y - half_height - half_thickness),
            (box_x - half_width, box_y + half_height + half_thickness),
            "left"
        ))
    
    if opening_side != 'right':  # Right wall
        wall_configs.append((
            (box_x + half_width, box_y - half_height - half_thickness),
            (box_x + half_width, box_y + half_height + half_thickness),
            "right"
        ))
    
    # Create the walls
    for start_pos, end_pos, description in wall_configs:
        # Create static body
        wall_body = pymunk.Body(body_type=pymunk.Body.STATIC)
        
        # Create line segment shape
        wall_shape = pymunk.Segment(wall_body, start_pos, end_pos, wall_thickness/2)
        wall_shape.friction = 0.0
        wall_shape.elasticity = 0.0
        wall_shape.color = (100, 100, 100, 255)  # Gray color for walls
        
        # Add to space
        space.add(wall_body, wall_shape)
        walls.append((wall_body, wall_shape, description))
    
    return walls, box_bounds


def is_worm_outside_chamber(worm, box_bounds, exit_distance=30):
    """
    Check if a worm has completely exited the growth chamber.
    
    Args:
        worm: The worm object to check
        box_bounds: Dictionary containing chamber boundaries
        exit_distance: How far past the opening the worm must be to be considered "outside"
    
    Returns:
        True if worm should be removed, False otherwise
    """
    if not worm.bodies:
        return True
    
    opening_side = box_bounds['opening_side']
    
    # Check all segments of the worm
    for body in worm.bodies:
        x, y = body.position
        
        # Determine if this segment is still "inside" or close to the chamber
        if opening_side == 'right':
            # For right opening, worm is "outside" when all segments are far to the right
            if x <= box_bounds['right'] + exit_distance:
                return False  # At least one segment is still close to chamber
        elif opening_side == 'left':
            if x >= box_bounds['left'] - exit_distance:
                return False
        elif opening_side == 'top':
            if y >= box_bounds['top'] - exit_distance:
                return False
        elif opening_side == 'bottom':
            if y <= box_bounds['bottom'] + exit_distance:
                return False
    
    # If we get here, all segments are far from the chamber
    return True


def remove_worm_from_simulation(worm, space):
    """
    Safely remove a worm and all its components from the simulation.
    
    Args:
        worm: The worm object to remove
        space: The pymunk space
    """
    # Remove all joints first
    for joint in worm.joints:
        if joint in space.constraints:
            space.remove(joint)
    
    # Remove all bodies and shapes
    for body, shape in zip(worm.bodies, worm.shapes):
        if body in space.bodies:
            space.remove(body)
        if shape in space.shapes:
            space.remove(shape)
    
    # Clear the worm's lists
    worm.bodies.clear()
    worm.shapes.clear()
    worm.joints.clear()


class Worm:
    def __init__(
        self,
        space,
        start_pos,
        num_segments,
        segment_radius,
        segment_mass,
        group_id,
        growth_rate=5.0,
        max_length=30,
        min_length_after_division=10,
        max_length_variation=0.2,
        base_color=None,
        noise_strength=0.05,
        base_max_length=None,  # NEW: Track the original base length
        _from_division=False,
    ):
        self.space = space
        self.start_pos = start_pos
        self.segment_radius = segment_radius
        self.segment_mass = segment_mass
        self.growth_rate = growth_rate
        self.max_bend_angle = 0.01
        self.noise_strength = noise_strength

        self.group_id = group_id
        self.base_color = base_color if base_color else generate_color(group_id)

        # NEW: Store the original base max_length for consistent inheritance
        self.base_max_length = base_max_length if base_max_length is not None else max_length
        
        # Always randomize from the original base, not the parent's randomized value
        variation = self.base_max_length * max_length_variation
        random_max_len = np.random.uniform(
            self.base_max_length - variation, self.base_max_length + variation
        )
        self.max_length = max(min_length_after_division * 2, int(random_max_len))

        self.min_length_after_division = min_length_after_division
        self.max_length_variation = max_length_variation

        # Rest of the existing code...
        self.bodies = []
        self.shapes = []
        self.joints = []

        self.growth_accumulator = 0.0
        self.growth_threshold = self.segment_radius / 3
        self.joint_distance = self.segment_radius / 4
        self.joint_max_force = 290000 # can turn it  to 90,000 when the chamber is shorter

        if not _from_division:
            for i in range(num_segments):
                self._add_initial_segment(i == 0)
            self._update_colors()

    def _add_initial_segment(self, is_first):
        """
        Adds a single segment to the worm during initialization.
        """
        moment = pymunk.moment_for_circle(
            self.segment_mass, 0, self.segment_radius
        )
        body = pymunk.Body(self.segment_mass, moment)

        if is_first:
            body.position = self.start_pos
        else:
            prev_body = self.bodies[-1]
            # Keep growth perfectly straight
            offset = Vec2d(self.joint_distance, 0).rotated(prev_body.angle)
            body.position = prev_body.position + offset
            
            # NEW: Add tiny random positional noise to break determinism
            noise_x = np.random.uniform(-0.1, 0.1)
            noise_y = np.random.uniform(-0.1, 0.1)
            body.position += Vec2d(noise_x, noise_y)

        shape = pymunk.Circle(body, self.segment_radius)
        shape.friction = 0.0  # User change
        shape.filter = pymunk.ShapeFilter(group=self.group_id)

        self.space.add(body, shape)
        self.bodies.append(body)
        self.shapes.append(shape)

        if not is_first:
            prev_body = self.bodies[-2]

            anchor_on_prev = (self.joint_distance / 2, 0)
            anchor_on_curr = (-self.joint_distance / 2, 0)
            pivot = pymunk.PivotJoint(
                prev_body, body, anchor_on_prev, anchor_on_curr
            )
            pivot.max_force = self.joint_max_force
            self.space.add(pivot)
            self.joints.append(pivot)

            limit = pymunk.RotaryLimitJoint(
                prev_body, body, -self.max_bend_angle, self.max_bend_angle
            )
            limit.max_force = self.joint_max_force
            self.space.add(limit)
            self.joints.append(limit)

    def apply_noise(self, dt):
        """
        NEW: Apply small random forces to all segments to simulate environmental noise
        """
        for body in self.bodies:
            # Apply tiny random forces
            force_x = np.random.uniform(-self.noise_strength, self.noise_strength)
            force_y = np.random.uniform(-self.noise_strength, self.noise_strength)
            body.force += Vec2d(force_x, force_y)
            
            # Also apply tiny random torques
            torque = np.random.uniform(-self.noise_strength * 0.1, self.noise_strength * 0.1)
            body.torque += torque

    def grow(self, dt):
        """
        Grows the worm by extending the last segment until a new one can be added.
        """
        if len(self.bodies) >= self.max_length or len(self.bodies) < 2:
            return

        # User change: randomized growth
        self.growth_accumulator += (
            self.growth_rate * dt * np.random.uniform(0, 4)
        )
        last_pivot_joint = self.joints[-2]
        original_anchor_x = -self.joint_distance / 2
        last_pivot_joint.anchor_b = (
            original_anchor_x - self.growth_accumulator,
            0,
        )

        if self.growth_accumulator >= self.growth_threshold:
            pre_tail_body = self.bodies[-2]
            old_tail_body = self.bodies[-1]

            last_pivot_joint.anchor_b = (original_anchor_x, 0)

            stable_offset = Vec2d(self.joint_distance, 0).rotated(
                pre_tail_body.angle
            )
            old_tail_body.position = pre_tail_body.position + stable_offset
            old_tail_body.angle = pre_tail_body.angle

            moment = pymunk.moment_for_circle(
                self.segment_mass, 0, self.segment_radius
            )
            new_tail_body = pymunk.Body(self.segment_mass, moment)
            
            # Keep growth direction perfectly straight
            new_tail_offset = Vec2d(self.joint_distance, 0).rotated(
                old_tail_body.angle
            )
            new_tail_body.position = old_tail_body.position + new_tail_offset
            
            # NEW: Add tiny random positional noise to the new segment
            noise_x = np.random.uniform(-0.1, 0.1)
            noise_y = np.random.uniform(-0.1, 0.1)
            new_tail_body.position += Vec2d(noise_x, noise_y)

            new_tail_shape = pymunk.Circle(new_tail_body, self.segment_radius)
            new_tail_shape.friction = 0.0  # User change
            new_tail_shape.filter = pymunk.ShapeFilter(group=self.group_id)

            self.space.add(new_tail_body, new_tail_shape)
            self.bodies.append(new_tail_body)
            self.shapes.append(new_tail_shape)

            anchor_on_prev = (self.joint_distance / 2, 0)
            anchor_on_curr = (-self.joint_distance / 2, 0)
            new_pivot = pymunk.PivotJoint(
                old_tail_body, new_tail_body, anchor_on_prev, anchor_on_curr
            )
            new_pivot.max_force = self.joint_max_force
            self.space.add(new_pivot)
            self.joints.append(new_pivot)

            new_limit = pymunk.RotaryLimitJoint(
                old_tail_body,
                new_tail_body,
                -self.max_bend_angle,
                self.max_bend_angle,
            )
            new_limit.max_force = self.joint_max_force
            self.space.add(new_limit)
            self.joints.append(new_limit)

            self.growth_accumulator = 0.0
            self._update_colors()

    def divide(self, next_group_id):
        """
        If the worm is at max_length, it splits by transplanting its second
        half into a new Worm object, preserving orientation.
        """
        if len(self.bodies) < self.max_length:
            return None

        split_index = len(self.bodies) // 2
        if split_index < self.min_length_after_division or (
            len(self.bodies) - split_index
        ) < self.min_length_after_division:
            return None

        # Create daughter worm - pass the BASE max_length, not this worm's randomized one
        daughter_worm = Worm(
            self.space,
            self.bodies[split_index].position,
            0,
            self.segment_radius,
            self.segment_mass,
            next_group_id,
            self.growth_rate,
            self.base_max_length,  # FIXED: Pass the original base length
            self.min_length_after_division,
            self.max_length_variation,
            base_color=generate_color(next_group_id),
            noise_strength=self.noise_strength,
            base_max_length=self.base_max_length,  # NEW: Ensure base is preserved
            _from_division=True,
        )

        # Partition the mother's parts.
        daughter_worm.bodies = self.bodies[split_index:]
        daughter_worm.shapes = self.shapes[split_index:]
        daughter_worm.joints = self.joints[split_index * 2 :]

        for shape in daughter_worm.shapes:
            shape.filter = pymunk.ShapeFilter(group=next_group_id)

        connecting_joint = self.joints[(split_index - 1) * 2]
        connecting_limit = self.joints[(split_index - 1) * 2 + 1]
        self.space.remove(connecting_joint, connecting_limit)

        self.bodies = self.bodies[:split_index]
        self.shapes = self.shapes[:split_index]
        self.joints = self.joints[: (split_index - 1) * 2]

        self._update_colors()
        daughter_worm._update_colors()

        return daughter_worm

    def remove_tail_segment(self):
        """
        Safely removes the last segment of the worm.
        """
        if len(self.bodies) <= self.min_length_after_division:
            return

        tail_body = self.bodies.pop()
        tail_shape = self.shapes.pop()

        tail_joint = self.joints.pop()
        tail_limit = self.joints.pop()

        self.space.remove(tail_body, tail_shape, tail_joint, tail_limit)
        self._update_colors()

    def _update_colors(self):
        """
        Sets the head and tail colors based on the worm's base color.
        """
        if not self.shapes:
            return
            
        # NEW: Use the worm's base color with variations
        r, g, b = self.base_color
        
        # Body segments: use base color with alpha
        body_color = (r, g, b, 255)
        
        # Head: brighter version of base color
        head_color = (
            min(255, int(r * 1.3)),
            min(255, int(g * 1.3)),
            min(255, int(b * 1.3)),
            255
        )
        
        # Tail: darker version of base color
        tail_color = (
            int(r * 0.7),
            int(g * 0.7),
            int(b * 0.7),
            255
        )
        
        # Apply colors
        for shape in self.shapes:
            shape.color = body_color
        
        self.shapes[0].color = head_color   # Head
        self.shapes[-1].color = tail_color  # Tail


def main():
    """
    Initializes Pygame and Pymunk and runs the main simulation loop.
    """
    pygame.init()
    screen_width, screen_height = 1200, 800
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption("Dividing Worm Colony Simulation with Growth Chamber")

    space = pymunk.Space(threaded=True)
    space.threads = 2

    def setup_spatial_hash(space, colony):
        """Setup spatial hash with estimated parameters"""
        dim, count = estimate_spatial_hash_params(colony)
        space.use_spatial_hash(dim, count)
        print(f"Spatial hash enabled: dim={dim:.1f}, count={count}")
        return dim, count
    
    def estimate_spatial_hash_params(colony):
        """Estimate good spatial hash parameters for current colony size"""
        if not colony:
            return 36.0, 1000
        
        total_segments = sum(len(worm.bodies) for worm in colony)
        segment_radius = colony[0].segment_radius if colony else 15
        
        # dim: slightly larger than segment diameter for optimal performance
        dim = segment_radius * 2 * 1.2
        
        # count: ~10x total objects, with reasonable bounds
        count = max(1000, min(100000, total_segments * 10))
        print(f"Spatial hash updated: dim={dim:.1f}, count={count}")
        return dim, count
    

    # Add periodic updates in your main loop:
    frame_count = 0
    last_segment_count = 15  # Initial worm segments
    
    space.iterations = 60
    space.gravity = (0, 0)
    space.damping = 0.4
    
    # NEW: Camera and zoom variables
    zoom_level = 1.0
    camera_x, camera_y = 0, 0
    min_zoom = 0.1
    max_zoom = 5.0

    # Create a virtual surface for drawing
    virtual_surface = pygame.Surface((screen_width, screen_height))
    draw_options = pymunk.pygame_util.DrawOptions(virtual_surface)
    pymunk.pygame_util.positive_y_is_up = False

    # NEW: Create the growth chamber (open-ended box)
    chamber_x = screen_width / 2 - 200  # Position chamber to the left of center
    chamber_y = screen_height / 2
    chamber_width = 1500
    chamber_height = 50 # 60 is good for some amount of bend and width
    
    # Create the chamber walls (open on the right side for worms to flow out)
    chamber_walls, box_bounds = create_open_box(
        space, 
        chamber_x, 
        chamber_y, 
        chamber_width, 
        chamber_height, 
        wall_thickness=15, 
        opening_side='right'
    )
    
    print(f"Created growth chamber at ({chamber_x}, {chamber_y}) with {len(chamber_walls)} walls")

    colony = []
    next_group_id = 1
    total_worms_created = 0  # NEW: Track total worms created
    total_worms_removed = 0  # NEW: Track total worms removed

    # NEW: Position initial worm inside the growth chamber
    initial_worm_x = chamber_x - chamber_width/4  # Left side of chamber
    initial_worm_y = chamber_y
    
    initial_worm = Worm(
        space,
        start_pos=(initial_worm_x, initial_worm_y),
        num_segments=15,
        segment_radius=15,
        segment_mass=2,
        group_id=next_group_id,
        max_length=40,  # This is now the mean length
        base_color=generate_color(next_group_id),  # NEW: Set initial color
        noise_strength=0.1,  # NEW: Small environmental noise
    )
    colony.append(initial_worm)
    next_group_id += 1
    total_worms_created += 1

    # In your main() function, after creating initial_worm:
    setup_spatial_hash(space, colony)

    mouse_body = pymunk.Body(body_type=pymunk.Body.KINEMATIC)
    mouse_joint = None

    clock = pygame.time.Clock()
    running = True

    simulation_speed_multiplier = 10

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT or (
                event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE
            ):
                running = False
            # NEW: Handle mouse wheel for zooming
            elif event.type == pygame.MOUSEWHEEL:
                old_zoom = zoom_level
                if event.y > 0:  # Scroll up - zoom in
                    zoom_level = min(zoom_level * 1.1, max_zoom)
                elif event.y < 0:  # Scroll down - zoom out
                    zoom_level = max(zoom_level / 1.1, min_zoom)
                
                # Adjust camera to zoom towards mouse position
                mouse_x, mouse_y = pygame.mouse.get_pos()
                world_x = (mouse_x - screen_width/2) / old_zoom + camera_x
                world_y = (mouse_y - screen_height/2) / old_zoom + camera_y
                
                camera_x = world_x - (mouse_x - screen_width/2) / zoom_level
                camera_y = world_y - (mouse_y - screen_height/2) / zoom_level
                
            # NEW: Handle keyboard zoom controls as backup
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_EQUALS or event.key == pygame.K_PLUS:
                    zoom_level = min(zoom_level * 1.2, max_zoom)
                elif event.key == pygame.K_MINUS:
                    zoom_level = max(zoom_level / 1.2, min_zoom)
                elif event.key == pygame.K_r:  # Reset zoom and camera
                    zoom_level = 1.0
                    camera_x, camera_y = 0, 0
                elif event.key == pygame.K_j:  # NEW: Toggle joint visibility
                    if hasattr(draw_options, 'show_joints'):
                        draw_options.show_joints = not draw_options.show_joints
                    
            elif event.type == pygame.MOUSEBUTTONDOWN:
                # NEW: Convert mouse position to world coordinates
                mouse_x, mouse_y = event.pos
                world_x = (mouse_x - screen_width/2) / zoom_level + camera_x
                world_y = (mouse_y - screen_height/2) / zoom_level + camera_y
                pos = Vec2d(world_x, world_y)
                
                hit = space.point_query_nearest(pos, 5/zoom_level, pymunk.ShapeFilter())
                if hit is not None and hit.shape.body.body_type == pymunk.Body.DYNAMIC:
                    shape = hit.shape
                    rest_point = shape.body.world_to_local(pos)
                    mouse_joint = pymunk.PivotJoint(
                        mouse_body, shape.body, (0, 0), rest_point
                    )
                    mouse_joint.max_force = 100000
                    mouse_joint.error_bias = (1 - 0.15) ** 60
                    space.add(mouse_joint)
            elif event.type == pygame.MOUSEBUTTONUP:
                if mouse_joint is not None:
                    space.remove(mouse_joint)
                    mouse_joint = None

        # NEW: Update mouse body position in world coordinates        
        mouse_x, mouse_y = pygame.mouse.get_pos()
        world_mouse_x = (mouse_x - screen_width/2) / zoom_level + camera_x
        world_mouse_y = (mouse_y - screen_height/2) / zoom_level + camera_y
        mouse_body.position = (world_mouse_x, world_mouse_y)
        
        dt = 1.0 / 60.0

        for _ in range(simulation_speed_multiplier):
            newly_born_worms_map = {}

            for worm in colony[:]:
                worm.apply_noise(dt)  # NEW: Apply environmental noise
                worm.grow(dt)
                new_worm = worm.divide(next_group_id)
                if new_worm:
                    newly_born_worms_map[new_worm] = worm
                    next_group_id += 1
                    total_worms_created += 1

            if newly_born_worms_map:
                counter = 0
                for daughter, mother in newly_born_worms_map.items():
                    while True:
                        overlap_found = False
                        for daughter_shape in daughter.shapes:
                            query_result = space.shape_query(daughter_shape)

                            for info in query_result:
                                if info.shape in mother.shapes:
                                    mother.remove_tail_segment()
                                    overlap_found = True
                                    break
                            if overlap_found:
                                break

                        if not overlap_found:
                            break
                        counter += 1
                        if counter > 10000:
                            break

            colony.extend(newly_born_worms_map.keys())
            
            # NEW: Remove worms that have completely exited the chamber
            worms_to_remove = []
            for worm in colony:
                if is_worm_outside_chamber(worm, box_bounds, exit_distance=20):
                    worms_to_remove.append(worm)
            
            for worm in worms_to_remove:
                remove_worm_from_simulation(worm, space)
                colony.remove(worm)
                total_worms_removed += 1
            
            if worms_to_remove:
                print(f"Removed {len(worms_to_remove)} worms from simulation")
            
            space.step(dt)

        # NEW: Apply camera transform to draw options
        draw_options.transform = pymunk.Transform(
            a=zoom_level, b=0, c=0, d=zoom_level,
            tx=screen_width/2 - camera_x * zoom_level,
            ty=screen_height/2 - camera_y * zoom_level
        )

        # NEW: Hide joints when zoomed out, show when zoomed in
        if zoom_level < 0.8:  # Hide joints when zoomed out
            draw_options.flags = pymunk.pygame_util.DrawOptions.DRAW_SHAPES
        else:  # Show joints when zoomed in enough
            draw_options.flags = (
                pymunk.pygame_util.DrawOptions.DRAW_SHAPES |
                pymunk.pygame_util.DrawOptions.DRAW_CONSTRAINTS
            )

        # Clear both surfaces
        screen.fill((20, 30, 40))
        virtual_surface.fill((20, 30, 40))
        
        # Draw to virtual surface with transform
        space.debug_draw(draw_options)
        
        # Blit virtual surface to main screen
        screen.blit(virtual_surface, (0, 0))
        
        # NEW: Display comprehensive info
        font = pygame.font.Font(None, 36)
        zoom_text = font.render(f"Zoom: {zoom_level:.2f}x", True, (255, 255, 255))
        colony_text = font.render(f"Active Worms: {len(colony)}", True, (255, 255, 255))
        created_text = font.render(f"Total Created: {total_worms_created}", True, (255, 255, 255))
        removed_text = font.render(f"Total Removed: {total_worms_removed}", True, (255, 255, 255))
        help_text = font.render("Mouse wheel: Zoom, R: Reset, J: Toggle joints", True, (255, 255, 255))
        
        screen.blit(zoom_text, (10, 10))
        screen.blit(colony_text, (10, 50))
        screen.blit(created_text, (10, 90))
        screen.blit(removed_text, (10, 130))
        screen.blit(help_text, (10, 170))
        
        pygame.display.flip()
        clock.tick(60)

        frame_count += 1
    
        # Update spatial hash every 60 frames (1 second) if colony grew significantly
        if frame_count % 60 == 0:
            current_segment_count = sum(len(worm.bodies) for worm in colony)
            if current_segment_count > last_segment_count * 1.5:  # 50% growth
                dim, count = estimate_spatial_hash_params(colony)
                space.use_spatial_hash(dim, count)
                last_segment_count = current_segment_count

    pygame.quit()


if __name__ == "__main__":
    main()